# **Applied Statistics Problems Notebook**

This notebook contains my answers to each of the four problems given for the assessmnet of the Applied Statistics module.

The aim of the notebook is to show an understanding of the key principals of statistics and to demonstrate the following abilities, taken from the learning outcomes of the module:

- Describe the stochastic nature of real-world measurements.

- Source documentation to programmatically perform a statistical test.

- Select an appropriate statistical test to investigate a claim.

- Perform a statistical test on a data set.

Within this notebook `Code blocks` are used wherever code is referenced.

***
## Problem 1: Extending the Lady Tasting Tea

The Lady Tasting Tea experiment is a randomised experiment devised by Ronald Fisher. In the experiment, 8 randomly ordered cups of tea are prepared: 4 of those cups of tea are prepared with milk followed by tea and 4 get tea first followed by milk. A participant claims they can distinguish the 2 sets of cups of tea from one another without seeing them being prepared.

The null hypothesis of the experiment is that the subject drinking the tea is not able to distinguish whether the cup they are drinking from had the tea or milk poured in it first.

In this first problem I will extend the Lady Tasting Tea experiment so that there are 12 cups: 8 tea-first and 4 milk-first.

I will use `numpy` to randomly shuffle the cups many times and then calculate the probability of the participant correctly identifying all cups by chance.

Lets begin by importing the libraries relevant to this problem:

In [1]:
# Mathematical functions from the standard library.
# https://docs.python.org/3/library/math.html
import math

# Random selections.
# https://docs.python.org/3/library/random.html
import random

# Numerical structures and operations.
# https://numpy.org/doc/stable/reference/index.html#reference
import numpy as np

Now lets define the cups of tea that will be available in the set, in this version of the experiment there will be 4 with milk first and 8 with tea first.

In [2]:
# Number of cups of tea in total.
no_cups = 12

# Number of cups of tea with milk in first.
no_cups_milk_first = 4

# Number of cups of tea with tea in first.
no_cups_tea_first = 8

`Math.comb` is a function we can use to find out how many ways to choose k items from n items without repetition and without order.

We will use it here to find out how many ways there are to select four cups from 12.

In [3]:
# Number of ways of selecting four cups from 12.
ways = math.comb(no_cups, no_cups_milk_first)

# Show.
ways

495

Evidently there are 495 ways to select four cups from 12.

Now lets work out the probability that the participant in this experiment guesses the four cups correctly.

In [4]:
# Probability that she randomly selects the four correct cups.
p_all_correct = 1 / ways

# Show
p_all_correct

0.00202020202020202

We can now simulate the experiment a number of times (100000 times to give us a more accurate result).

In doing so we will count the number of times the participant guesses each cup correctly.

In [5]:
# Number of simulations
n_sims = 100000

# The true cups with milk in first (labelled 0–11 for 12 cups total)
true_milk = set(range(no_cups_milk_first))

# Using NumPy to randomly shuffle cups
rng = np.random.default_rng()
hits = 0

for _ in range(n_sims):
    cups = np.arange(no_cups)
    rng.shuffle(cups)
    guess = set(cups[:no_cups_milk_first])  # Participant’s guess
    if guess == true_milk:
        hits += 1

# Estimate the probability
simulated_p = hits / n_sims

print(f"Simulated probability of all correct: {simulated_p:.6f}")
print(f"Exact probability: {p_all_correct:.6f}")

Simulated probability of all correct: 0.001770
Exact probability: 0.002020


In this simulation, I used `Numpy` to randomly shuffle 12 cups (8 tea-first and 4 milk-first) to estimate the probability of a participant correctly identifying all cups by chance.

In this extended 12-cup experiment the probability of correctly identifying all cups by chance is 1 in 495 (p is roughly 0.002) compared to 1 in 70 (p is roughly 0.0143) in the original 8-cup version. This means that perfectly guessing the cups is far more unlikely in the extended experiment.

This indicates that the extended experiment is more stringent, so we would not need to lower the p-value threshold, and could even relax it slightly since achieving a perfect result is already far less likely by chance.

***
## Problem 2: Normal Distribution

For the second task I will generate 100,000 samples of size 10 from the standard normal distribution. For each sample, I will compute the standard deviation with ddof=1 (sample SD) and with ddof=0 (population SD).

Following this, I will plot histograms of both sets of values on the same axes with transparency and speak about the differenves between the two sets.

To begin we will need to import `matplotlib` to help with plotting.

In [6]:
# Plotting.
# https://matplotlib.org/stable/contents.html
import matplotlib.pyplot as plt

Now lets generate 100,000 samples of size 10 from the standard normal distribution.

In [ ]:
# drawing 100,000 samples
n_samples = 100_000

# determining the size of each sample
sample_size = 10     

For each sample, I will compute the standard deviation with ddof=1 (sample SD) and with ddof=0 (population SD)

In [8]:
# Shape: (100000, 10)
samples = np.random.randn(n_samples, sample_size)

# show
samples

array([[-0.24467922,  1.22343848,  0.02440525, ...,  0.06867439,
        -0.41722268,  0.54032482],
       [-0.68691477,  0.40861722, -2.48948666, ..., -0.38395648,
        -0.61051712,  1.66751264],
       [ 1.67280674, -0.24676371,  0.25333045, ...,  1.97362687,
         0.62004334, -0.43225072],
       ...,
       [-0.31583205, -1.01762646, -0.26418761, ...,  0.21062421,
         1.44400847,  0.67287327],
       [-2.08155532,  0.35021012,  2.62411628, ...,  0.19314236,
         0.39891245, -0.52737806],
       [-0.27223835, -0.62285825,  0.22520545, ..., -1.07537487,
         0.24713662, -0.08275378]])

plot histograms of both sets of values on the same axes with transparency and speak about the differenves between the two sets

***
## Problem 3: t-Tests

For the third task,

***
## Problem 4: ANOVA

In this final task

***


## References

| Author                                 | Title                                                                                                         | Blurb                           |
| :---                                   |    :----:                                                                                                     |          :---:                  |
| Wikimedia Commons | [ Chardin, 'A Lady Taking Tea' ](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4b/Chardin_-_A_Lady_Taking_Tea%2C_1735.jpg/960px-Chardin_-_A_Lady_Taking_Tea%2C_1735.jpg) | Cover image used in this repositories README, Jean-Baptiste-Siméon Chardin: A Lady Taking Tea (1735). |
| Ronald Fisher | The Design of Experiments (1935) | Ronald Fishers book on experimental design which contains the Lady Tasting Tea experiment. |
| Python Documentation | [Python math.comb documentation](https://docs.python.org/3/library/math.html#math.comb) | Pythons math.comb documentation. |
| Python Documentation | [Python's math module documentation](https://docs.python.org/3/library/math.html) | Info on python's math module. |
| NumPy API Reference | [NumPy Reference Manual](https://numpy.org/doc/stable/reference/index.html#reference) | This reference manual details functions, modules, and objects included in NumPy, describing what they are and what they do. |

***

## End

***